## Assignment Topic:

In this project, you will build a regression model using the Keras library to model the same data about concrete compressive strength that we used in labs 3.p In this course project, you will build a regression model using the deep learning Keras library, and then you will experiment with increasing the number of training epochs and changing number of hidden layers and you will see how changing these parameters impacts the performance of the model.

### Increasing the Number of Training Epochs - 2

***

## Data Preprocessing

In [1]:
import pandas as pd
import numpy as np

In [2]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.

Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

(1030, 9)
So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

In [6]:
concrete_data_columns = concrete_data.columns

target = concrete_data["Strength"]
predictors = concrete_data.drop(["Strength", "Age"], axis=1)

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569


In [10]:
n_cols = predictors_norm.shape[1]  # number of predictors
n_cols

7

***

## Keras

Recall from the videos that Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras.

In [11]:
import warnings
warnings.filterwarnings("ignore")
import keras

In [12]:
from keras.models import Sequential
from keras.layers import Dense

## Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [13]:
# define regression model
def regression_model():
    # 1-Create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # 2-Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

In [16]:
model = regression_model()

In [17]:
epochs = 120
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/120
23/23 [==============================] - 0s 2ms/step - loss: 29148.1445
Epoch 2/120
23/23 [==============================] - 0s 2ms/step - loss: 1629.3865
Epoch 3/120
23/23 [==============================] - 0s 2ms/step - loss: 397.6479
Epoch 4/120
23/23 [==============================] - 0s 2ms/step - loss: 282.4565
Epoch 5/120
23/23 [==============================] - 0s 2ms/step - loss: 243.9864
Epoch 6/120
23/23 [==============================] - 0s 1ms/step - loss: 222.6523
Epoch 7/120
23/23 [==============================] - 0s 2ms/step - loss: 211.3314
Epoch 8/120
23/23 [==============================] - 0s 1ms/step - loss: 203.4998
Epoch 9/120
23/23 [==============================] - 0s 1ms/step - loss: 199.1552
Epoch 10/120
23/23 [==============================] - ETA: 0s - loss: 148.668 - 0s 1ms/step - loss: 193.0871
Epoch 11/120
23/23 [==============================] - 0s 2ms/step - loss: 189.8641
Epoch 12/120
23/23 [==============================] - 0s 1ms/step -

23/23 [==============================] - 0s 1ms/step - loss: 145.3873
Epoch 97/120
23/23 [==============================] - 0s 1ms/step - loss: 144.6744
Epoch 98/120
23/23 [==============================] - 0s 1ms/step - loss: 144.2365
Epoch 99/120
23/23 [==============================] - 0s 1ms/step - loss: 146.0400
Epoch 100/120
23/23 [==============================] - 0s 1ms/step - loss: 144.4123
Epoch 101/120
23/23 [==============================] - 0s 1ms/step - loss: 145.5309
Epoch 102/120
23/23 [==============================] - 0s 1ms/step - loss: 151.6246
Epoch 103/120
23/23 [==============================] - 0s 1ms/step - loss: 142.2427
Epoch 104/120
23/23 [==============================] - 0s 1ms/step - loss: 145.9662
Epoch 105/120
23/23 [==============================] - 0s 1ms/step - loss: 155.3216
Epoch 106/120
23/23 [==============================] - 0s 1ms/step - loss: 142.5934
Epoch 107/120
23/23 [==============================] - 0s 1ms/step - loss: 153.5686
Epoch 108

In [18]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

10/10 [==============================] - 0s 1ms/step - loss: 153.2946


153.2946319580078

***

### Mean Squared Error

In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(f"Mean: {mean}\nStandard Deviation: {standard_deviation}")

Mean: 153.29465370223676
Standard Deviation: 0.0
